In [23]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import os
import matplotlib.pyplot as plt
%matplotlib inline
print("PACKAGES LOADED")

PACKAGES LOADED


In [24]:
mnist = input_data.read_data_sets('data/',one_hot=True)
trainimg = mnist.train.images
trainlabel = mnist.train.labels
testimg = mnist.test.images
testlabel = mnist.test.labels
print("MNIST loaded")

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
MNIST loaded


In [25]:
#DEFINE MODEL
#notwork topologies
n_input = 784
n_channel = 64#컨볼루션 필터 개수
n_classes = 10

#inputs and outputs
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

#network parameters
stddev = 0.1
weights = {
    'c1':tf.Variable(tf.random_normal([3,3,1,n_channel],stddev=stddev)),#convolution layer
    'c2':tf.Variable(tf.random_normal([3,3,n_channel,128],stddev=stddev)),
    'd1':tf.Variable(tf.random_normal([7*7*128,n_classes],stddev=stddev)),#denselayer
}
biases = {
    'c1':tf.Variable(tf.random_normal([n_channel],stddev=stddev)),
    'c2':tf.Variable(tf.random_normal([128],stddev=stddev)),
    'd1':tf.Variable(tf.random_normal([n_classes],stddev=stddev)),
}
print("NETWORK READY")


NETWORK READY


In [39]:
#MODEL
def CNN(_x, _w, _b):
    # RESHAPE(현재 벡터형태라서 텐서형태로 바꿔줘야됨.)
    _x_r = tf.reshape(_x, shape=[-1, 28, 28, 1])#앞에 몇갠지 몰라서 -1 여기선 784해줘도돼, 그다음이 28x28x1
    # CONVOLUTION
    _conv1 = tf.nn.conv2d(_x_r, _w['c1'], strides=[1, 1, 1, 1], padding='SAME')
    # ADD BIAS
    _conv2 = tf.nn.bias_add(_conv1, _b['c1'])
    # RELU
    _conv3 = tf.nn.relu(_conv2)
    #MAX-POOl
    _pool = tf.nn.max_pool(_conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
 
    # SECOND CONV LAYER
    _temp = tf.nn.conv2d(_pool, _w['c2'], strides=[1, 1, 1, 1], padding='SAME')
    _temp = tf.nn.bias_add(_temp, _b['c2'])
    _temp = tf.nn.relu(_temp)
    _temp = tf.nn.max_pool(_temp, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    # VECTORIZE
    _dense = tf.reshape(_temp, [-1, _w['d1'].get_shape().as_list()[0]])#한줄로 핀다
    # DENSE
    _logit = tf.add(tf.matmul(_dense, _w['d1']), _b['d1'])
    _out = {
        'x_r' : _x_r, 'conv1' : _conv1, 'conv2' : _conv2, 'conv3' : _conv3,
        'pool' : _pool, 'dense' : _dense, 'logit' : _logit
}
    return _out

# PREDICTION
cnnout = CNN(x, weights, biases)

# LOSS AND OPTIMZER
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=cnnout['logit']))
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)#cost를 최소화하는 방향으로 아담 최적화함수를사용한다
corr = tf.equal(tf.argmax(cnnout['logit'], 1), tf.argmax(y, 1))
accr = tf.reduce_mean(tf.cast(corr, "float"))

# INITIALIZER
init = tf.global_variables_initializer()
print ("FUNCTION READY")

FUNCTION READY


In [40]:
#SAVER
savedir="nets/cnn_mnist_simple/"
saver=tf.train.Saver(max_to_keep=3)#??최근 3개 모델만 저장
save_step=4#얜뭐지?
if not os.path.exists(savedir):
    os.makedirs(savedir)
print("SAVER READY")

SAVER READY


In [42]:
#RUN(멀티퍼셉트론꺼 복붙)
#PARAMETERS
training_epochs=20
batch_size=100
display_step=4

#LAUNCH THE GRAPH
sess = tf.Session()
sess.run(init)

#OPTIMIZE
for epoch in range(training_epochs):
    avg_cost=0.
    total_batch=int(mnist.train.num_examples/batch_size)
    #ITERATION
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds={x:batch_xs, y:batch_ys}
        sess.run(optm, feed_dict=feeds)
        avg_cost+=sess.run(cost, feed_dict=feeds)
    avg_cost = avg_cost/total_batch
    
    #DISPLAY
    if(epoch+1)%display_step==0:
        print("EPOCH : %03d/%03d cost: %.9f"%(epoch,training_epochs,avg_cost))
        feeds={x:batch_xs, y:batch_ys}
        train_acc=sess.run(accr, feed_dict=feeds)
        print("TRAIN ACCURACY: %.3f" %(train_acc))
        feeds={x:mnist.test.images[:100,:], y:mnist.test.labels[:100,:]}
        test_acc=sess.run(accr,feed_dict=feeds)
        print("TEST ACCURACY: %.3f" %(test_acc))
    
    #SAVE
    if(epoch+1)%save_step==0:
        savename=savedir+"net-"+str(epoch+1)+".ckpt"
        saver.save(sess.savename)
        print("[%s] SAVED"%(savename))
print("OPTIMIZATION FINISHED")

KeyboardInterrupt: 

In [ ]:
#RESTORE
do_restore = 1
if do_restore ==1:
    sess = tf.Session()
    epoch = 4
    savename = savedir+"net-"+str(epoch)+".ckpt"
    saver.restore(sess,savename)
    print("NETWORK RESTORED")
else:
    print("DO NOTHING")

In [ ]:
#중간과정 엿보기
input_r=sess.run(cnnout['x_r'],feed_dict={c:trainimg[0:1,:]})
conv1=sess.run(cnnout['conv1'],feed_dict={x:trainimg[0:1,:]})
conv2=sess.run(cnnout['conv2'],feed_dict={x:trainimg[0:1,:]})
conv3=sess.run(cnnout['conv3'],feed_dict={x:trainimg[0:1,:]})
pool=sess.run(cnnout['pool'],feed_dict={x:trainimg[0:1,:]})
dense=sess.run(cnnout['dense'],feed_dict={x:trainimg[0:1,:]})
out=sess.run(cnnout['logit'],feed_dict={x:trainimg[0:1,:]})

In [ ]:
#input
print("Size of 'input_r' is %s"%(input_r.shape,))
label=np.argmax(trainlabel[0,:])
print("Label is %d"%(label))

#plot
plt.matshow(input_r[0,:,:,0],cmap=plt.get_cmap('gray'))
plt.title("Label of this image is "+str(label)+"")
plt.colorbar()
plt.show()

In [ ]:
#conv
print("Size of 'conv1' is %s"%(conv1.shape,))#이거하면 사이즈 (1,28,28,64)라서 64갠데 3개만 보여주겠다는거
for i in range(3):
    plt.matshow(conv1[0,:,:,i],cmap=plt.get_cmap('gray'))
    plt.title(str(i)+"th conv1")
    plt.colorbar()
    plt.show()

In [ ]:
#conv + bias. 모양은 (1,28,28,64)로 똑같은데 컬러바의 값 range를 바꿔줌
print("Size of 'conv2' is %s"%(conv2.shape,))#이거하면 사이즈 (1,28,28,64)라서 64갠데 3개만 보여주겠다는거
for i in range(3):
    plt.matshow(conv2[0,:,:,i],cmap=plt.get_cmap('gray'))
    plt.title(str(i)+"th conv2")
    plt.colorbar()
    plt.show()

In [ ]:
#conv+bias+relu. sparse represion이 가능하게 해줌 렐루 거침으로써 마이너스 값들은 0으로 되고 나머지값들도 1보다 작은 값으로 만들어버림.이래서 바이어스가 중요한게 만약 바이어스가 -100이었으면 모든값이 렐루 거치고 나서 다0되어버리겠지
print("Size of 'conv3' is %s"%(conv3.shape,))#이거하면 사이즈 (1,28,28,64)라서 64갠데 3개만 보여주겠다는거
for i in range(3):
    plt.matshow(conv3[0,:,:,i],cmap=plt.get_cmap('gray'))
    plt.title(str(i)+"th conv3")
    plt.colorbar()
    plt.show()

In [ ]:
#POOL
print("Size of 'pool' is %s"%(pool.shape,))#이거하면 사이즈 (1,14,14,64)라서 64갠데 3개만 보여주겠다는거 course한(뭉툭한) 형태
for i in range(3):
    plt.matshow(pool[0,:,:,i],cmap=plt.get_cmap('gray'))
    plt.title(str(i)+"th pool")
    plt.colorbar()
    plt.show()

In [ ]:
#DENSE
print("Size of 'DENSE' is %s"%(dens.shape,))#이거하면 사이즈 (1,6272)
print("Size of 'OUT' is %s"%(out.shape,))#이거하면 사이즈 (1,10)
plt.matshow(out,cmap=plt.get_cmap('gray'))
plt.title("OUT")
plt.colorbar()
plt.show()

In [ ]:
#CONVOLUTION FILTER
wc1=sess.run(weights['c1'])
print("SIZE OF 'wc1' IS %s" %(wc1.shape,))
for i in range(3):
    plt.matshow(wc1[0,:,:,i],cmap=plt.get_cmap('gray'))
    plt.title(str(i)+"th conv filter")
    plt.colorbar()
    plt.show()